In [1]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝/synthetic_regression_data.csv')
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,target
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,1.308878
1,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,-1.071908
2,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,-0.603686
3,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,-1.961344
4,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861,2.749412


In [2]:
X = df.drop('target', axis=1)  # 학습용 입력 데이터
y = df['target']               # 학습용 출력 데이터

print("입력 데이터 X:")
print(X.head())

print("\n출력 데이터 y:")
print(y.head())

입력 데이터 X:
   feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0   0.496714  -0.138264   0.647689   1.523030  -0.234153  -0.234137   
1  -0.469474   0.542560  -0.463418  -0.465730   0.241962  -1.913280   
2  -1.012831   0.314247  -0.908024  -1.412304   1.465649  -0.225776   
3  -0.544383   0.110923  -1.150994   0.375698  -0.600639  -0.291694   
4  -0.013497  -1.057711   0.822545  -1.220844   0.208864  -1.959670   

   feature_7  feature_8  
0   1.579213   0.767435  
1  -1.724918  -0.562288  
2   0.067528  -1.424748  
3  -0.601707   1.852278  
4  -1.328186   0.196861  

출력 데이터 y:
0    1.308878
1   -1.071908
2   -0.603686
3   -1.961344
4    2.749412
Name: target, dtype: float64


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"훈련 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")



훈련 데이터 크기: (80, 8)
테스트 데이터 크기: (20, 8)


In [8]:
X_train

pandas.core.frame.DataFrame

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32) #X_train이 df 형태이므로 values를 통해 값만 가져옴
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # (N,) -> (N, 1)로 reshape
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

class SimpleRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleRegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x=self.fc1(x)
        x = self.relu(x)
        x=self.fc2(x)
        x = self.relu(x)
        x=self.fc3(x)
        return x

input_dim = X_train.shape[1]  # (batch,dim)여기서는 80,8
model = SimpleRegressionModel(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # 학습 모드로 설정
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()       # 기울기 초기화
        outputs = model(X_batch)    # 모델 예측
        loss = criterion(outputs, y_batch)  # 손실 계산
        loss.backward()             # 역전파
        optimizer.step()            # 가중치 업데이트

    # 매 10 에포크마다 손실 출력
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 9. 테스트 데이터로 모델 평가
model.eval()  # 평가 모드로 설정
with torch.no_grad():  # 평가 시에는 기울기를 계산하지 않음
    test_loss = 0
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')

Epoch [10/100], Loss: 0.1567
Epoch [20/100], Loss: 0.1646
Epoch [30/100], Loss: 0.0250
Epoch [40/100], Loss: 0.0028
Epoch [50/100], Loss: 0.0311
Epoch [60/100], Loss: 0.0531
Epoch [70/100], Loss: 0.0086
Epoch [80/100], Loss: 0.0218
Epoch [90/100], Loss: 0.0031
Epoch [100/100], Loss: 0.0120
Test Loss: 1.4546
